In [6]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import pandas as pd
import requests

## Lectura de datos

In [62]:


# Obtener la fecha de inicio del usuario
#fecha_inicio_str = input("Ingrese la fecha de inicio (DD-MM-YYYY): ")
#fecha_inicio = datetime.strptime(fecha_inicio_str, "%d-%m-%Y")

# Obtener la fecha de fin del usuario
#fecha_fin_str = input("Ingrese la fecha de fin (DD-MM-YYYY): ")
#fecha_fin = datetime.strptime(fecha_fin_str, "%d-%m-%Y")

# Definir las dos fechas de inicio y fin
fecha_inicio = datetime(2020, 2, 1)
fecha_fin = datetime(2020, 4, 20)

# obtener primera fecha
link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+fecha_inicio.strftime("%m-%d-%Y")+".csv"
df = pd.read_csv(link, index_col=0)

## remplazar columna que varia en DataFrame
contry = df.filter(regex="Country").columns.tolist()
df.rename(columns={contry[0]:"Country_Region"}, inplace=True)
# sumar un dia
fecha_inicio += timedelta(days=1)

# Recorrer todas las fechas en el intervalo
while fecha_inicio <= fecha_fin:

    try:

        link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+fecha_inicio.strftime("%m-%d-%Y")+".csv"
        df2 = pd.read_csv(link,index_col=0)
        
        ## remplazar columna que varia en DataFrame
        contry = df2.filter(regex="Country").columns.tolist()
        df2.rename(columns={contry[0]:"Country_Region"}, inplace=True)

        df = pd.concat([df,df2],ignore_index=True)
        fecha_inicio += timedelta(days=1)
    except:
        pass

df.reset_index(inplace=True)


### 2. Información sobre el conjunto de datos (número de columnas, tipos de datos, entre otros). 

In [63]:
filas = str(df.shape[0])
columna = str(df.shape[1])

print("el conjunto de dato tiene "+filas+" filas y "+columna+" columnas")
print("\n\n")

print("Tipos de datos")
print(df.dtypes)




el conjunto de dato tiene 12048 filas y 14 columnas



Tipos de datos
index                    int64
Admin2                  object
Province_State          object
Country_Region          object
Last_Update             object
Lat                    float64
Long_                  float64
Confirmed                int64
Deaths                   int64
Recovered              float64
Active                 float64
Combined_Key            object
Incident_Rate          float64
Case_Fatality_Ratio    float64
dtype: object


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12048 entries, 0 to 12047
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                12048 non-null  int64  
 1   Admin2               9816 non-null   object 
 2   Province_State       11511 non-null  object 
 3   Country_Region       12048 non-null  object 
 4   Last_Update          12048 non-null  object 
 5   Lat                  11775 non-null  float64
 6   Long_                11775 non-null  float64
 7   Confirmed            12048 non-null  int64  
 8   Deaths               12048 non-null  int64  
 9   Recovered            26 non-null     float64
 10  Active               26 non-null     float64
 11  Combined_Key         12048 non-null  object 
 12  Incident_Rate        11773 non-null  float64
 13  Case_Fatality_Ratio  11927 non-null  float64
dtypes: float64(6), int64(3), object(5)
memory usage: 1.3+ MB


### 3. Información sobre los datos faltantes /inconsistentes dentro del conjunto de datos

In [125]:
datos_faltantes = df.isnull().sum()
datos_faltantes

array(['Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana',
       'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati',
       'Korea, South', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia',
       'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein',
       'Lithuania', 'Luxembourg', 'MS Zaandam', 'Madagascar', 'Malawi',
       'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands',
       'Mauritania', 'Mauritius', 'Mexico', 'Micronesia', 'Moldova',
       'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique',
       'Namibia', 'Nepal', 'Netherlands', 'New Zealand', 'Nicaragua',
       'Niger', 'Nigeria', 'North Macedonia', 'Norway', 'Oman',
       'Pakistan', 'Palau', 'Pana

###  4. Número de casos confirmados, fallecidos, recuperados y activos por país/región

In [44]:

df["Recovered"].fillna(0,inplace=True)
df["Active"].fillna(0,inplace=True)
df["Province_State"].fillna("sin provi",inplace=True)

## replazar datos inconsistentes
df["Active"] = df["Active"].astype(int)
df["Recovered"] = df["Recovered"].astype(int)

#df["Province_State"].replace(np.nan, "Sin informacion" ,inplace=True)

## agrupar por pais/Reguib
Group4 = df.groupby(["Country_Region"])[["Confirmed","Deaths","Recovered","Active"]].sum()
Group4 

,Confirmed,Deaths,Recovered,Active
Country_Region,,,,
Afghanistan,3374942,149471,0,0
Albania,5323302,67982,0,0
Algeria,5200525,133963,0,0
Andorra,740018,2962,0,0
Angola,1969238,37945,0,0
...,...,...,...,...
West Bank and Gaza,11830308,104444,0,0
Winter Olympics 2022,9063,0,0,0
Yemen,229776,41529,0,0


### 5. Número de casos confirmados, fallecidos y recuperados por país/región y provincia/estado

In [43]:
## agrupar por pais y privincie
Group5 = df.groupby(["Country_Region","Province_State"])[["Confirmed","Deaths","Recovered","Active"]].sum()
Group5

Confirmed  \
Country_Region Province_State                                            
Australia      Australian Capital Territory                     816374   
               New South Wales                                23676591   
               Northern Territory                               556192   
               Queensland                                      9658389   
               South Australia                                 2546611   
...                                                                ...   
United Kingdom Saint Helena, Ascension and Tristan da Cunha         80   
               Scotland                                       24939156   
               Turks and Caicos Islands                         116140   
               Unknown                                               0   
               Wales                                          15925724   

                                                             Deaths  \
Country_Region Province_State                                         
Australia      Australian Capital Territory                     601   
               New South Wales                                33338   
               Northern Territory                               161   
               Queensland                                      7043   
               South Australia                                 2905   
...                                                             ...   
United Kingdom Saint Helena, Ascension and Tristan da Cunha       0   
               Scotland                                      264490   
               Turks and Caicos Islands                         700   
               Unknown                                         5423   
               Wales                                         194181   

                                                             Recovered  Active  
Country_Region Province_State                                                   
Australia      Australian Capital Territory                        0.0     0.0  
               New South Wales                                     0.0     0.0  
               Northern Territory                                  0.0     0.0  
               Queensland                                          0.0     0.0  
               South Australia                                     0.0     0.0  
...                                                                ...     ...  
United Kingdom Saint Helena, Ascension and Tristan da Cunha        0.0     0.0  
               Scotland                                            0.0     0.0  
               Turks and Caicos Islands                            0.0     0.0  
               Unknown                                             0.0     0.0  
               Wales                                               0.0     0.0  

[617 rows x 4 columns]

### Número de casos confirmados, fallecidos, recuperados en China según provincia/estado.

In [10]:
df[df["Country_Region"] == "China"].groupby(["Province_State"])[["Confirmed","Deaths","Recovered","Active"]].sum()

,Confirmed,Deaths,Recovered,Active
Province_State,,,,
Anhui,3033,18,0,0
Beijing,4187,27,0,0
Chongqing,1850,18,0,0
Fujian,4489,3,0,0
Gansu,1236,6,0,0
Guangdong,13078,24,0,0
Guangxi,2865,6,0,0
Guizhou,484,6,0,0
Hainan,573,18,0,0


### 7. Número de casos confirmados, fallecidos, recuperados en USA según provincia/estado. 

In [11]:
df[df["Country_Region"] == "US"].groupby(["Province_State"])[["Confirmed","Deaths","Recovered","Active"]].sum()

,Confirmed,Deaths,Recovered,Active
Province_State,,,,
Alabama,3530081,52892,0,0
Alaska,641595,3295,0,0
American Samoa,479,0,0,0
Arizona,5410380,79719,0,0
Arkansas,2249255,30106,0,0
California,24208633,246151,0,0
Colorado,3972077,35928,0,0
Connecticut,2046165,30211,0,0
Delaware,718755,7719,0,0


### 8. Número de fallecidos por país

In [13]:
df.groupby(["Country_Region"])["Deaths"].sum()

Country_Region
Afghanistan             22489
Albania                 10127
Algeria                 19900
Andorra                   441
Angola                   5629
                        ...  
West Bank and Gaza      15796
Winter Olympics 2022        0
Yemen                    6218
Zambia                  11712
Zimbabwe                15961
Name: Deaths, Length: 201, dtype: int64

### 9. Listado de países que no cuentan con recuperados. 

In [66]:
Recovered_sum = df.groupby(["Country_Region"])["Recovered"].sum()
notRecovered = Recovered_sum[Recovered_sum == 0]
notRecovered

Country_Region
 Azerbaijan                       0
Antarctica                        0
Aruba                             0
Bahamas, The                      0
Botswana                          0
Cape Verde                        0
Cayman Islands                    0
Channel Islands                   0
Curacao                           0
Czech Republic                    0
East Timor                        0
Faroe Islands                     0
Gambia, The                       0
Greenland                         0
Guadeloupe                        0
Guam                              0
Guernsey                          0
Jersey                            0
Kiribati                          0
Korea, North                      0
MS Zaandam                        0
Martinique                        0
Mayotte                           0
Nauru                             0
North Ireland                     0
Palau                             0
Palestine                         0
Papua New Gui

### 10. Número de fallecidos por coronavirus por país.

In [68]:
Recovered_sum = df.groupby(["Country_Region"])["Recovered"].sum()
Recovered_sum

Country_Region
 Azerbaijan                          0
Afghanistan                        915
Albania                           4072
Algeria                           9870
Andorra                           2043
                                  ... 
Winter Olympics 2022                 0
Yemen                                0
Zambia                             384
Zimbabwe                            10
occupied Palestinian territory       0
Name: Recovered, Length: 242, dtype: int64

### 12. Top 10 de los países con más casos de coronavirus confirmados indicando su última fecha de actualización.